In [1]:
import os
import sys
import string
import pickle
import tarfile
import numpy as np
import pandas as pd
from gensim import corpora
from gensim.models import ldamodel, Phrases, phrases
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem.snowball import SnowballStemmer


class classifytweet:
    def __init__(self, model_files='model_files/'):
        """
        tweet_body: the individual tweet
        model_files: the location for the model files. leave the model files folder as is
                     unless you have changed the location of all the model files.
        """
        self.model_files = model_files
        #self.model, self.corpus, self.dictionary, self.phraser, self.valence_mean, self.arousal_mean, self.valence_sd, self.arousal_sd = self.load_model()
        self.load_model()
        self.n = len(self.dictionary.items())
        self.word_map = {v:k for k,v in self.dictionary.items()}
        print("Model items loaded and classifier initialized!")
        
    def load_model(self):
        """
        Loads the model, corpus, and dictionary.
        """
        # extract tarfiles
        for f in os.listdir(self.model_files):
            if f.endswith('.gz'):
                tar = tarfile.open(self.model_files + f, "r:gz")
                tar.extractall(path=self.model_files)
                tar.close()

        # load model, corpus, and dictionary objects
        fnames = [fn for fn in os.listdir(self.model_files) if '.gensim' in fn]
        self.model = ldamodel.LdaModel.load(self.model_files + fnames[0])
        self.corpus = corpora.MmCorpus(self.model_files + 'unigrams_corpus.mm')
        self.dictionary = corpora.Dictionary.load(self.model_files + 'unigrams_dictionary.pkl')
        self.model.id2word = self.dictionary
        self.phraser = phrases.Phrases.load(self.model_files + 'document_phraser.pkl')
        for f in ['unigrams_dictionary.pkl', 'unigrams_corpus.mm', 'unigrams_corpus.mm.index', 'NB_vectorizer.pkl', 'NB_sentiment_model.pkl']:
            fnames.append(f)
        
        # load the valence and arousal arrays
        self.valence_mean = pickle.load(open(self.model_files + 'valence_mean.pkl', 'rb'))
        self.arousal_mean = pickle.load(open(self.model_files + 'arousal_mean.pkl', 'rb'))
        self.valence_sd = pickle.load(open(self.model_files + 'valence_sd.pkl', 'rb'))
        self.arousal_sd = pickle.load(open(self.model_files + 'arousal_sd.pkl', 'rb'))
        
        # load the MinMaxScaler for the transforming the scores
        self.base_outrage_scaler = None
        self.expanded_outrage_scaler = None
        self.valence_scaler = None
        self.arousal_scaler = None
        self.emoji_scaler = None
        self.topic_valence_scaler = pickle.load(open(self.model_files + 'topic_valence_scaled.pkl', 'rb'))
        self.topic_arousal_scaler = pickle.load(open(self.model_files + 'topic_arousal_scaled.pkl', 'rb'))
        
        # load the Naive Bayes sentiment model
        try:
            self.nb_model = pickle.load(open(self.model_files + 'NB_sentiment_model.pkl', 'rb'))
        except:
            self.nb_model = pickle.load(open(self.model_files + 'NB_sentiment_model.pkl', 'rb'), encoding='latin1')
        try:
            self.nb_vectorizer = pickle.load(open(self.model_files + 'NB_vectorizer.pkl', 'rb'))
        except:
            self.nb_vectorizer = pickle.load(open(self.model_files + 'NB_vectorizer.pkl', 'rb'), encoding='latin1')
        
        # load the outrage dictionaries
        self.outrage_list = pd.read_csv(self.model_files + 'outrage_dictionary_stemmed.csv', header=None)
        self.exp_outrage_list = pd.read_csv(self.model_files + 'expanded_outrage_dictionary_stemmed.csv', header=None)

        # cleanup the unzipped files
        for f in fnames:
            os.remove(self.model_files + f)

    def prepare_tweet(self, tweet):
        """
        Turn that unstructured text into sweet, sweet, "cleaned" up tokens!
        """
        self.tweet = tweet
        stemmer = SnowballStemmer("english")
        tokenizer = TweetTokenizer()
        self.tweet_tokenized = tokenizer.tokenize(self.tweet)
        try:
            self.tweet_tokenized = [unicode(y.encode("utf-8"), errors='ignore') for y in self.tweet_tokenized]
            self.stemmed = [stemmer.stem(y) for y in self.tweet_tokenized]
        except:
            #tweet_tokenized = [y.encode("utf-8") for y in tweet_tokenized]
            self.stemmed = [stemmer.stem(y) for y in self.tweet_tokenized]
        
        #self.text_bigrams = [' '.join(self.stemmed[i:]) for i in range(2)]
        #self.text_bigrams=list(bigrams(self.stemmed))
        #self.text_bigrams=["%s %s" % x for x in self.text_bigrams]
        #self.text_bigrams.extend(self.stemmed)

        keep = set(['!','?'])
        stop = set(stopwords.words('english'))
        remove = set([x for x in list(string.punctuation) if x not in keep])
        stop.update(remove)
        stop.update(['',' ','  '])
        stemmed = [d for d in self.stemmed if d not in stop]
        self.phrased = list(self.phraser[[stemmed]])[0]

        #print('Phrased representation: "' + ' '.join(self.phrased) + '"')
        #return None
    
    def get_valence_score(self):
        """
        Creates the valence and arousal score for the tweet.
        """
        tweet_arr = np.zeros(self.n)
        for word in set(self.phrased) & set(self.word_map.keys()):
            tweet_arr[self.word_map[word]] = 1.
        mean = tweet_arr * self.valence_mean
        sd = tweet_arr * self.valence_sd
        total_sd = np.sum(sd) * tweet_arr
        with np.errstate(divide='ignore'):
            sd_ratio = total_sd / sd
            sd_ratio[sd == 0] = 0
        sd_weight = sd_ratio / np.sum(sd_ratio)
        
        if np.sum(mean*sd_weight) == np.nan:
            self.valence_score = 0
        else:
            self.valence_score = np.sum(mean*sd_weight)
        
        return self.valence_score

    def get_arousal_score(self):
        """
        Creates the valence and arousal score for the tweet.
        """
        tweet_arr = np.zeros(self.n)
        for word in set(self.phrased) & set(self.word_map.keys()):
            tweet_arr[self.word_map[word]] = 1.
        mean = tweet_arr * self.arousal_mean
        sd = tweet_arr * self.arousal_sd
        total_sd = np.sum(sd) * tweet_arr
        with np.errstate(divide='ignore'):
            sd_ratio = total_sd / sd
            sd_ratio[sd == 0] = 0
        sd_weight = sd_ratio / np.sum(sd_ratio)
        
        if np.sum(mean*sd_weight) == np.nan:
            self.arousal_score = 0
        else:
            self.arousal_score = np.sum(mean*sd_weight)
        
        return self.arousal_score

    def get_sentiment_score(self):
        """
        Weights the posititive/negative sentiment of the tweet.
        """
        vectorized = self.nb_vectorizer.transform(self.stemmed)
        self.sentiment_score = np.average(1 - self.nb_model.predict_proba(vectorized)[:,1])

        return self.sentiment_score

    def get_topics(self):
        """
        Extract the topics from the tweet using the LDA model.
        """
        return self.model.get_document_topics(self.model.id2word.doc2bow(self.phrased), per_word_topics=False)

    def get_emoji_count(self):
        """
        Count the Mad! faces.
        """
        positives = ['<U+0082>', '<U+008D>']
        outrage = ['<U+00A0>', '<U+00A1>', '<U+00A4>', '<U+00A9>']
        positive_score = sum([y in positives for y in self.tweet_tokenized])
        outrage_score = sum([y in outrage for y in self.tweet_tokenized])
        self.emoji_count = outrage_score-positive_score
        return self.emoji_count

    def get_base_outrage_count(self):
        """
        Get the number of outrage words in the tweet.
        """
        self.base_outrage_count = 0
        for i in self.stemmed:
            self.base_outrage_count += len(set(i) & set(self.outrage_list))
        return self.base_outrage_count

    def get_expanded_outrage_count(self):
        """
        Get the number of outrage words in the tweet.
        """
        self.expanded_outrage_count = 0
        for i in self.stemmed:
            self.expanded_outrage_count += len(set(i) & set(self.exp_outrage_list))
        return self.expanded_outrage_count

    def get_outrage_score(self):
        """
        Uses the results of each of the index measures to create one score.
        .20 outrage dict
        .15 expanded outrage dict
        .15 arousal
        .13 valence
        .11 sentiment
        .10 emoji
        .08 topic valence
        .08 topic arousal
        """
        self.topics = self.get_topics()
        topic_valence_score = 0
        topic_arousal_score = 0
        for tup in self.topics:
            topic_valence_score += self.topic_valence_scaler[tup[0]] * tup[1]
            topic_arousal_score += self.topic_valence_scaler[tup[0]] * tup[1]
            
        scores = np.array([
            self.get_base_outrage_count(),
            self.get_expanded_outrage_count(),
            self.get_arousal_score(),
            self.get_valence_score(),
            self.get_sentiment_score(),
            self.get_emoji_count(),
            topic_valence_score,
            topic_arousal_score
            ])
        weights = np.array([0.2, 0.15, 0.15, 0.13, 0.11, 0.10, 0.08, 0.08])

        self.outrage_meter = np.sum(scores*weights)
        return self.outrage_meter

In [2]:
#import tweetclassifier
from time import time
import datetime

t0 = time()
tweeter = classifytweet()
elapsed = time() - t0
print("Took %s to initialize." % (str(datetime.timedelta(seconds=elapsed))))

Model items loaded and classifier initialized!
Took 0:00:23.199991 to initialize.


In [3]:
tweeter.prepare_tweet('Climate Change is wreaking havoc in #Paris tonight!')

In [4]:
tweeter.get_valence_score()

//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:126: RuntimeWarning: invalid value encountered in true_divide
//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:128: RuntimeWarning: invalid value encountered in true_divide


nan

In [5]:
tweeter.get_arousal_score()

//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:145: RuntimeWarning: invalid value encountered in true_divide
//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:147: RuntimeWarning: invalid value encountered in true_divide


nan

In [26]:
tweet_arr = np.zeros(tweeter.n)
for word in set(tweeter.stemmed) & set(tweeter.word_map.keys()):
    tweet_arr[tweeter.word_map[word]] = 1.
    print(word, tweeter.valence_mean[tweeter.word_map[word]])
mean = tweet_arr * tweeter.valence_mean
sd = tweet_arr * tweeter.valence_sd
total_sd = np.sum(sd) * tweet_arr
print(np.sum(mean))
with np.errstate(divide='ignore'):
    sd_ratio = total_sd / sd
    sd_ratio[sd == 0] = 0
sd_weight = sd_ratio / np.sum(sd_ratio)

tweeter.valence_score = np.sum(mean*sd_weight)

climat 5.8
! 0.0
wreak 3.5
#pari 0.0
chang 5.09
havoc 3.85
tonight 0.0
18.24


//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


In [24]:
set(tweeter.phrased) & set(tweeter.word_map.keys())

{'!', '#pari', 'climat_chang', 'tonight', 'wreak_havoc'}

In [25]:
set(tweeter.stemmed) & set(tweeter.word_map.keys())

{'!', '#pari', 'chang', 'climat', 'havoc', 'tonight', 'wreak'}

In [6]:
tweeter.get_topics()

[(5, 0.24355570076495781),
 (16, 0.34282610972905425),
 (32, 0.25528485617265501)]

In [7]:
for tup in tweeter.get_topics():
    print("Topic",tup[0],tweeter.model.print_topic(tup[0],topn=15))

Topic 5 0.034*"wed_day" + 0.033*"climat_chang" + 0.032*"citi" + 0.019*"truth" + 0.017*"wed_cake" + 0.012*"statement" + 0.008*"worst" + 0.008*"..." + 0.007*"@ap" + 0.007*"fool" + 0.007*"18" + 0.007*"abt" + 0.006*"@johnkerri" + 0.006*"ha" + 0.005*"aka"
Topic 16 0.483*"!" + 0.023*"thank" + 0.011*"climat_chang" + 0.008*"wow" + 0.007*"love" + 0.005*"..." + 0.005*"oh" + 0.005*"get" + 0.005*"go" + 0.005*"great" + 0.004*"wors" + 0.004*"damn" + 0.004*"yes" + 0.004*"time" + 0.003*"omg"
Topic 32 0.030*"climat_chang" + 0.023*"#pari" + 0.016*"greater_threat" + 0.014*"environ" + 0.013*"100" + 0.011*"declar" + 0.010*"key" + 0.010*"send" + 0.008*"..." + 0.008*"billion" + 0.008*"@realjameswood" + 0.007*"8" + 0.007*"time" + 0.006*"excel" + 0.006*"yeah"


In [8]:
tweeter.get_sentiment_score()

0.49889298242521807

In [9]:
tweeter.get_emoji_count()

0

In [10]:
tweeter.get_base_outrage_count()

0

In [11]:
tweeter.get_expanded_outrage_count()

0

In [12]:
tweeter.get_outrage_score()

//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:145: RuntimeWarning: invalid value encountered in true_divide
//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:147: RuntimeWarning: invalid value encountered in true_divide
//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:126: RuntimeWarning: invalid value encountered in true_divide
//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:128: RuntimeWarning: invalid value encountered in true_divide


nan

In [13]:
[(float(x.split('*')[0]),x.split('*')[1][1:-1]) for x in tweeter.model.print_topics(num_topics=60, num_words=30)[0][1].split(" + ")]

[(0.332, 'global_warm'),
 (0.013, 'new_york'),
 (0.013, '...'),
 (0.009, 'via_@dailycal'),
 (0.009, 'time'),
 (0.008, 'caus'),
 (0.007, 'predict'),
 (0.005, 'hoax'),
 (0.004, 'climat_chang'),
 (0.004, 'theori'),
 (0.004, 'excit'),
 (0.004, '#isi'),
 (0.004, 'obama'),
 (0.004, '#tcot'),
 (0.004, 'antarct_ice'),
 (0.003, '#forecast'),
 (0.003, 'develop'),
 (0.003, 'say'),
 (0.003, 'claim'),
 (0.003, 'crowd'),
 (0.003, 'trend'),
 (0.003, 'summer'),
 (0.003, 'gain_ice'),
 (0.003, 'send_forc'),
 (0.003, 'metorologist_fight'),
 (0.003, 'https://t.co/gprzfbifbm_obama'),
 (0.003, 'new'),
 (0.003, 'denial'),
 (0.003, 'enrich_famili'),
 (0.003, 'speed')]

In [14]:
tweeter.topics

[(5, 0.24338928017259367),
 (16, 0.34250665049894941),
 (32, 0.25577073599512407)]